In [1]:
# import and define functions for the ElisPix project
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os
from scipy import ndimage


def crop_image(img, x1, y1, x2, y2):
    # Ensure coordinates are within bounds
    h, w = img.shape[:2]
    x1, y1 = max(0, int(x1)), max(0, int(y1))
    x2, y2 = min(w, int(x2)), min(h, int(y2))

    # Crop the image
    cropped_img = img[y1:y2, x1:x2]
    return cropped_img


def smearArray(rowsum, percentsmear = 0.00125):
    smearedrowsum = np.zeros(len(rowsum))
    smearlength = int(len(rowsum) * percentsmear)
    temprowsum = np.zeros(len(rowsum)+2*smearlength)
    temprowsum[smearlength:-smearlength] = rowsum
    print(len(smearedrowsum))
    for i in range(smearlength):
        smearedrowsum = smearedrowsum + temprowsum[i:i+len(rowsum)] / (smearlength*2)
        # Now the other side
        smearedrowsum = smearedrowsum + temprowsum[-i-len(rowsum)-1:-i-1] / (smearlength*2)
    return smearedrowsum


def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result


def findCutpoints(thresh, thresholddivider = 200, percentsmear_val = 0.00125, row_or_col = 'row'):
    picstarts = []
    picends = []
    if row_or_col == 'row':
        # print('row')
        rowsum = np.sum(thresh, axis=1)
    else:
        rowsum = np.sum(thresh, axis=0)
        # print('col')
    rowsum = smearArray(rowsum, percentsmear_val)
    rowsum_orig = rowsum
    thresholdvalue = np.mean(rowsum)/thresholddivider
    loopflag = True
    plt.close()
    plt.figure()
    cutvalue = 0
    while loopflag:
        # check if rowsum is all zeros or all nonzeros
        if np.all(rowsum > thresholdvalue) or np.all(rowsum < thresholdvalue):
            loopflag = False
        else:
            # check if there are any non zeros in the rowsum array
            if np.any(rowsum > thresholdvalue):
                picstarts.append(np.argmax(rowsum > thresholdvalue)+cutvalue)
                rowsum = rowsum[np.argmax(rowsum > thresholdvalue):]
                cutvalue = picstarts[-1]
            if np.any(rowsum < thresholdvalue):
                picends.append(np.argmax(rowsum < thresholdvalue)+cutvalue)
                rowsum = rowsum[np.argmax(rowsum < thresholdvalue):]
                cutvalue = picends[-1]

    return picstarts, picends, rowsum_orig


def find_top_bottom_corners(thresh):
    # Find all non-zero pixel coordinates
    non_zero_y, non_zero_x = np.where(thresh > 0)

    # Find top corner (minimum y-coordinate)
    top_idx = np.argmin(non_zero_y)
    top_corner = (non_zero_x[top_idx], non_zero_y[top_idx])

    # Find bottom corner (maximum y-coordinate)
    bottom_idx = np.argmax(non_zero_y)
    bottom_corner = (non_zero_x[bottom_idx], non_zero_y[bottom_idx])

    return top_corner, bottom_corner


def rotate_point(x, y, height, width, angle):
    # Convert angle to radians
    angle_rad = np.deg2rad(angle)
    
    # Define rotation matrix
    rotation_matrix = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad)],
        [np.sin(angle_rad), np.cos(angle_rad)]
    ])
    
    # Subtract the center of the image
    x_centered = x - width / 2
    y_centered = y - height / 2
    
    # Apply the rotation
    new_x_centered, new_y_centered = np.dot(rotation_matrix, [x_centered, y_centered])
    
    # Add back the center of the image
    new_x = new_x_centered + width / 2
    new_y = new_y_centered + height / 2
    
    return new_x, new_y


def intricate_angle_finder(currthresh):
    angles = []
    # flip currthresh left to right
    currthreshflipped = np.fliplr(currthresh)
    # flip currthresh up and down
    currthreshflipped = np.flipud(currthreshflipped)
    for i in range(10,90,1):
        n = i/100
        try:
            angles.append(find_rotation_angle(currthresh, startpercent = n, awaydistance = 0.05, enddistance = 0.05, skips = 2))
            angles.append(find_rotation_angle(currthreshflipped, startpercent = n, awaydistance = 0.05, enddistance = 0.05, skips = 2))
        except:
            continue
    return np.median(angles), angles


def find_rotation_angle(currthresh, startpercent = 0.8, awaydistance = 0.025, enddistance = 0.075, skips = 5):
    ylength = currthresh.shape[0]

    # # Choose starting % from top
    # startpercent = 0.8
    # # choose starting distance from startpercent from top
    # awaydistance = 0.025
    # # choose ending distance from startpercent
    # enddistance = 0.075
    # # choose how many points to skip
    # skips = 5

    ystart = int(ylength * startpercent)
    yupstart = int(ylength * (startpercent - awaydistance))
    yupend = int(ylength * (startpercent - awaydistance - enddistance))
    up_mean = (yupend+yupstart)/2
    ydownstart = int(ylength * (startpercent + awaydistance))
    ydownend = int(ylength * (startpercent + awaydistance + enddistance))
    down_mean = (ydownstart+ydownend)/2

    row_indices_25 = np.arange(yupend, yupstart, skips)
    first_white_pixels_25 = np.argmax(currthresh[row_indices_25, :], axis=1)
    mean_x25 = np.mean(first_white_pixels_25)

    row_indices_75 = np.arange(ydownstart, ydownend , skips)
    first_white_pixels_75 = np.argmax(currthresh[row_indices_75, :], axis=1)
    mean_x75 = np.mean(first_white_pixels_75)

    # find the slope of the line between the two points
    slope = (mean_x75 - mean_x25) / (down_mean - up_mean)

    # find the angle of the line
    angle = np.arctan(slope) * 180 / np.pi

    return angle


def remove_white_border(img2):
    # Convert image to grayscale
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to create a mask of the non-white pixels
    _, thresh3 = cv2.threshold(gray2, 250, 255, cv2.THRESH_BINARY_INV)

    thresholddivider_val = 200
    percentsmear_val = 0.005

    # find cutpoints rowwise
    _, _, rowsum = findCutpoints(thresh3, thresholddivider_val, percentsmear_val, 'row')
    _, _, colsum = findCutpoints(thresh3, thresholddivider_val, percentsmear_val, 'col')

    row_threshold = np.mean(rowsum)
    col_threshold = np.mean(colsum)

    startofimg = np.where(rowsum > row_threshold)[0][0]
    endofimg = np.where(rowsum > row_threshold)[0][-1]
    startofimgside = np.where(colsum > col_threshold)[0][0]
    endofimgside = np.where(colsum > col_threshold)[0][-1]

    return startofimg, endofimg, startofimgside, endofimgside




In [46]:
# Read in megaimage
picfoldername = r'D:\ElisPix\testfolder'
savefolder = r'D:\ElisPix\savefolder'

# list all files in folder
piclist = os.listdir(picfoldername)

# loop through all the megaimages to find the subimages
for picnum in range(len(piclist)):
    print(f'Working on {piclist[picnum]}: {picnum+1} of {len(piclist)}')
    # read in current pic as array
    img = mpimg.imread(picfoldername + '\\' + piclist[picnum])

    # change to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # apply threshold
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # apply threshold
    # ret, thresh = cv2.threshold(gray, 225, 255, cv2.THRESH_BINARY_INV)

    thresholddivider_val = 200
    percentsmear_val = 0.00125
    # find cutpoints rowwise
    picstarts_row, picends_row, _ = findCutpoints(thresh, thresholddivider_val, percentsmear_val, 'row')

    # adjust such that we have the same number of starts and ends and that the first value is a start
    if picstarts_row[0]>picends_row[0]:
        picstarts_row = [0] + picstarts_row
    if picends_row[-1]<picstarts_row[-1]:
        picends_row = picends_row + thresh.shape[0]

    Imagetop = dict() # dictionary to save the image coordinates of the pictures
    Imagecoord = dict() # dictionary to save the image coordinates of the pictures

    # loop through all starts and ends and save the image
    for i in range(len(picstarts_row)):
        # find cutpoints columnwise
        picstarts_col, picends_col, _ = findCutpoints(thresh[picstarts_row[i]:picends_row[i],:], thresholddivider_val, percentsmear_val, 'col')
        # adjust such that we have the same number of starts and ends and that the first value is a start
        if picstarts_col[0]>picends_col[0]:
            picstarts_col = [0] + picstarts_col
        if picends_col[-1]<picstarts_col[-1]:
            picends_col = picends_col + thresh.shape[1]
        # loop through all starts and ends and save the image
        for j in range(len(picstarts_col)):
            Imagecoord[f'Img_row{int(i+1)}_col{int(j+1)}'] = [picstarts_row[i],picends_row[i],picstarts_col[j],picends_col[j]]
            # save if the image top corner is on the left or right side
            Imagetop[f'Img_row{int(i+1)}_col{int(j+1)}'] = 'left' if picstarts_col[j] < thresh.shape[1]/2 else 'right'
    
    print(f'Found {len(Imagecoord)} images in {piclist[picnum]}')
    # loop through all images in the megaimage and save them
    for i in range(len(Imagecoord)):
        currentkey = list(Imagecoord.keys())[i]
        print(f'Working on {currentkey} from {piclist[picnum]}')
        currimg = img[Imagecoord[currentkey][0]:Imagecoord[currentkey][1],Imagecoord[currentkey][2]:Imagecoord[currentkey][3]]
        currthresh = thresh[Imagecoord[currentkey][0]:Imagecoord[currentkey][1],Imagecoord[currentkey][2]:Imagecoord[currentkey][3]]
        
        # To find the rotation angle we find the slope of the line between two x,y points
        # angle = find_rotation_angle(currthresh)
        angle, _ = intricate_angle_finder(currthresh)
        print(angle)

        # Find the corners before rotation
        fixsmear = int(percentsmear_val/2)
        if Imagetop[currentkey] == 'left':
            x_top, y_top = fixsmear, fixsmear
            x_bottom, y_bottom = currthresh.shape[1]-fixsmear, currthresh.shape[0]-fixsmear
        else:
            x_top, y_top = currthresh.shape[1]-fixsmear, fixsmear
            x_bottom, y_bottom = fixsmear, currthresh.shape[0]-fixsmear
        
        # Rotate the image
        # invert currimg such that black is white and white is black
        currimg = cv2.bitwise_not(currimg)
        rotated_img = ndimage.rotate(currimg, -angle)
        # invert currimg such that black is white and white is black
        rotated_img = cv2.bitwise_not(rotated_img)

        # currimg = cv2.cvtColor(currimg, cv2.COLOR_BGR2RGB)
        currimg = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2RGB)

        # rotate clockwize by 90 degrees
        currimg = cv2.rotate(currimg, cv2.ROTATE_90_CLOCKWISE)

        # crop the image by removing white border
        startofimg, endofimg, startofimgside, endofimgside = remove_white_border(currimg)
        currimg = currimg[startofimg:endofimg, startofimgside:endofimgside]
        
        # use cv2 to save image
        cv2.imwrite(savefolder + '\\' + piclist[picnum] + '_' + currentkey + '.png', currimg)

c:\Python311\Lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (143208000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


KeyboardInterrupt: 